In [ ]:
# import libraries
! pip install kaggle 

In [ ]:
#import DBB (dwnload from kaggle)
! kaggle datasets download ankitbansal06/retail-orders -f orders.csv    

In [2]:
#unzipp the DBB zip
import zipfile
zip_ref = zipfile.ZipFile('orders.csv.zip') #method pour 
zip_ref.extractall()  #si tu veux ouvrir ailleurs -> give the path here
zip_ref.close()

In [ ]:
#read data from the file and handle null values
import pandas as pd

In [ ]:
df = pd.read_csv('orders.csv')
df.head(15) #take a look

In [ ]:
df.shape

In [ ]:
## 3. change the non-values for NaN
df['Ship Mode'].unique() # montre les types- attirbutes de la colonne 

In [ ]:
#utiliser na_values methode
df = pd.read_csv('orders.csv', na_values= ['Not Available', 'unknown'])
df['Ship Mode'].unique()

In [ ]:
#fix the colome names (1to1): 
df.rename(columns={'Order Id' : 'order_id'})

In [ ]:
#rename nton
df.columns

In [ ]:
#df.columns.str.lower()
df.columns = df.columns.str.lower()
df

In [ ]:
#replacer hiphens
#df.columns.str.replace(' ','_')
df.columns = df.columns.str.replace(' ','_')
df

In [ ]:
###calculate new columns :  discount , sale price and profit

#df['list_price']*df['discount_percent']*.01

df['discount'] = df['list_price']*df['discount_percent']*.01

#df['list_price']-df['discount']
df['sale_price'] = df['list_price']-df['discount']

#df['sale_price']-df['cost_price']
df['profit'] = df['sale_price']-df['cost_price']

df.head(6)

In [ ]:
# Print the column information
df.info()

In [ ]:
#convert order date from object data type to datetime
df['order_date']=pd.to_datetime(df['order_date'],format="%Y-%m-%d")
df.head(6)

In [ ]:
#convert order date from object data type to datetime (order_date):
#df.drop(columns=['level_0','index','list_price','cost_price','discount_percent'],inplace=True) #attention à inplace=true
df.head(6)


In [ ]:
df.info()

In [58]:
#load the data into sql server using replace option
import sqlalchemy as sal


In [61]:
## TO DO

## connect to my sqlserver (is : DESKTOP-MT2MGIU  from sqlserver)  i.e. engine = sal.create_engine('mssql://ANKIT\SQLEXPRESS/master?driver=ODBC+DRIVER+17+FOR+SQL+SERVER')
# 1.on va creer une connection 'engine' et 2. le connector mssql (ou postre ou mysql si nécessaire) et 3. master est la base à la quelle je veux me connecter 
#  /master?driver=ODBC+DRIVER+17+FOR+SQL+SERVER vient de sqlserver bdd Master
#pip install pyodbc
# # driver : in windows rechercher : ODBC 

engine = sal.create_engine('mssql://DESKTOP-MT2MGIU/master?driver=ODBC+DRIVER+17+FOR+SQL+SERVER')
conn=engine.connect()




In [ ]:
#load the data into sql server using append option
df.to_sql('df_orders', con=conn , index=False, if_exists = 'append')

In [ ]:
/*to  get sharp with the DBB, drop the table, provide the rigth dLL adn change 'append' by 'replace'*/
CREATE TABLE df_orders(
 [order_id] int PRIMARY KEY,         
 [order_date] date,
 [ship_mode] VARCHAR(20),        
 [segment] VARCHAR(20),        
 [country] VARCHAR(20),        
 [city] VARCHAR(20),        
 [state] VARCHAR(20),        
 [postal_code] VARCHAR(20),         
 [region] VARCHAR(20),        
 [category] VARCHAR(20),        
 [sub_category] VARCHAR(20),        
 [product_id] VARCHAR(20),        
 [quantity] int,         
 [discount] DECIMAL(7,2),       
 [sale_price] DECIMAL(7,2),       
 [profit] DECIMAL(7,2)
)

In [ ]:
#load the data into sql server using append option
df.to_sql('df_orders', con=conn , index=False, if_exists = 'replace')

In [ ]:
df.columns

In [ ]:
SELECT TOP 10 * FROM df_orders;


--find top 10 highest reveue generating products 
SELECT TOP 10 product_id, SUM(sale_price) as sales
FROM df_orders
GROUP BY product_id
ORDER BY sales DESC;


--find top 5 highest selling products in each region
WITH fir1 AS(
	SELECT product_id, region, SUM(sale_price) as sales
	FROM df_orders
	GROUP BY region, product_id
) SELECT * FROM  (SELECT *, 
RANK()OVER(PARTITION BY region ORDER BY sales desc) as rnk
FROM fir1)


/*----------------*/
with cte as (
select region,product_id,sum(sale_price) as sales
from df_orders
group by region,product_id)

select * from 
(
select *
, row_number() over(partition by region order by sales desc) as rn
from cte) A
where rn<=5;

--find month over month growth comparison for 2022 and 2023 sales eg : jan 2022 vs jan 2023
--for each category which month had highest sales 
--which sub category had highest growth by profit in 2023 compare to 2022

In [ ]:

--find month over month growth comparison for 2022 and 2023 sales eg : jan 2022 vs jan 2023
with cte as (
select year(order_date) as order_year,month(order_date) as order_month,
sum(sale_price) as sales
from df_orders
group by year(order_date),month(order_date)
--order by year(order_date),month(order_date)
	)
select order_month
, sum(case when order_year=2022 then sales else 0 end) as sales_2022
, sum(case when order_year=2023 then sales else 0 end) as sales_2023
from cte 
group by order_month
order by order_month





--for each category which month had highest sales 
with cte as (
select category,format(order_date,'yyyyMM') as order_year_month
, sum(sale_price) as sales 
from df_orders
group by category,format(order_date,'yyyyMM')
--order by category,format(order_date,'yyyyMM')
)
select * from (
select *,
row_number() over(partition by category order by sales desc) as rn
from cte
) a
where rn=1






--which sub category had highest growth by profit in 2023 compare to 2022
with cte as (
select sub_category,year(order_date) as order_year,
sum(sale_price) as sales
from df_orders
group by sub_category,year(order_date)
--order by year(order_date),month(order_date)
	)
, cte2 as (
select sub_category
, sum(case when order_year=2022 then sales else 0 end) as sales_2022
, sum(case when order_year=2023 then sales else 0 end) as sales_2023
from cte 
group by sub_category
)
select top 1 *
,(sales_2023-sales_2022)
from  cte2
order by (sales_2023-sales_2022) desc